<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/fixed_income/spot_forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.optimize import minimize
import numpy as np

In [2]:
# Example
num = 5     # number of bonds

# bond 1 - 5
maturity = [2, 4, 6, 8, 10]
coupon_rate = [0.04, 0.04, 0.04, 0.04, 0.04]
yld = [0.02, 0.03, 0.04, 0.05, 0.06]
bonds = [{"maturity": 2, "coupon": 0.04, "yld": 0.02},
      {"maturity": 4, "coupon":  0.04, "yld": 0.03},
      {"maturity": 6, "coupon":  0.04, "yld": 0.04},
      {"maturity": 8, "coupon":  0.04, "yld": 0.05},
      {"maturity": 10, "coupon":  0.04, "yld": 0.06}]

# max periods
maxmaturity = np.max([bond['maturity'] for bond in bonds])
max_n = 2 * maxmaturity

In [8]:
# Yields, spot rates, and forward rates are computed with semi-annual compounding
def objective(bonds, spots):
    # error
    error = []
    for bond in bonds:
        coupon = 100 * bond['coupon'] / 2
        yld_2 = bond['yld'] / 2
        n = int(2 * bond['maturity'])
        p = coupon * np.sum((1 + yld_2) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + yld_2) ** n
        spot = np.array(spots[:n]) / 2
        phat = coupon * np.sum((1 + spot[:n]) ** np.arange(-1, -n - 1, -1)) + 100 / (1 + spot[n - 1]) ** n
        error.append(np.log(phat / p))        
    sse = np.sum([e ** 2 for e in error])

    # forward rates
    future_factors = (1+spots/2)**np.arange(1,len(spots)+1)
    change_logs = np.diff(np.log(future_factors))
    f = 2 * (np.exp(change_logs)-1)
    forwards = np.concatenate(([spots[0]], f))

    diffs = np.sum(np.diff(np.log(1+forwards/2))**2)
    return sse + 0.5*diffs

# Spot rates are yields of hypothetical zero-coupon bonds
spot_rate = minimize(lambda x: objective(bonds, x), [0.05]*max_n).x
print('The spot rate for the maturity of 10 years is {:.2%}'.format(spot_rate[-1]))

The spot rate for the maturity of 10 years is 6.42%


In [11]:
print('The yield for the 10-year maturity bond is {:.2%}'.format(bonds[-1]['yld']))

The yield for the 10-year maturity bond is 6.00%


In [12]:
# Forward rates are the rates that could be locked in for forward loans by trading the hypothetical zero-coupon bonds
future_factors = (1+spot_rate/2)**np.arange(1,len(spot_rate)+1)
change_logs = np.diff(np.log(future_factors))
f = 2 * (np.exp(change_logs)-1)
forwards_rate = np.concatenate(([spot_rate[0]], f))
print('The spot rate for the maturity of 10 years is {:.2%}'.format(forwards_rate[-1]))

The spot rate for the maturity of 10 years is 11.59%
